In [55]:
import pandas as pd
# Caminho do arquivo
url_imp_mun_2023 = r'\\wsl.localhost\Ubuntu\home\edcarlos\Data-Analytics-Comercio-Exterior\datalake\landing\importacao_mun_2023.csv' #foi nomeado dessa forma porque não encontrou o arquivo porque está no wsl2
# Lendo o CSV para um DataFrame
df = pd.read_csv(url_imp_mun_2023, sep=',')
# Exibir as primeiras linhas do DataFrame
df.head()

,cd_sh4,cd_pais,sg_ufmun,cd_mun,kg_liquido,vl_dolarfob,data_importacao_mun
0,3104,676,go,5305109,35717261,15254781,2023-05-01
1,6104,858,sp,3415004,8189,255925,2023-12-01
2,8543,399,sp,3419071,19,12156,2023-02-01
3,7326,249,sp,3413504,18,3859,2023-04-01
4,8542,161,sc,4209102,202,51196,2023-06-01


In [56]:
# Caminho do arquivo
url_ncm_sh = r'\\wsl.localhost\Ubuntu\home\edcarlos\Data-Analytics-Comercio-Exterior\datalake\raw_data\NCM_SH.csv' #foi nomeado dessa forma porque não encontrou o arquivo porque está no wsl2
# Lendo o CSV para um DataFrame
df = pd.read_csv(url_ncm_sh, sep=';',  encoding='latin1') #processo de transformar dados em um formato específico para permitir que esses dados sejam armazenados ou transmitidos de forma eficiente
# Exibir as primeiras linhas do DataFrame
df.head()

,CO_SH6,NO_SH6_POR,NO_SH6_ESP,NO_SH6_ING,CO_SH4,NO_SH4_POR,NO_SH4_ESP,NO_SH4_ING,CO_SH2,NO_SH2_POR,NO_SH2_ESP,NO_SH2_ING,CO_NCM_SECROM,NO_SEC_POR,NO_SEC_ESP,NO_SEC_ING
0,10110,"Animais vivos das espécies cavalar, asinina e ...","Caballos y asnos, reproductores de raza pura",Pure-bred breeding horses and asses,101,"Cavalos, asininos e muares, vivos","Caballos, asnos, mulos y burdéganos, vivos","Live horses, asses, mules and hinnies",1,Animais vivos,Animales vivos,Live animals,I,Animais vivos e produtos do reino animal,Animales vivos y productos del reino animal,Lives animals; Animal products
1,10111,"Cavalos reprodutores, de raça pura",Caballos reproductores de raza pura,Pure-bred breeding horses,101,"Cavalos, asininos e muares, vivos","Caballos, asnos, mulos y burdéganos, vivos","Live horses, asses, mules and hinnies",1,Animais vivos,Animales vivos,Live animals,I,Animais vivos e produtos do reino animal,Animales vivos y productos del reino animal,Lives animals; Animal products
2,10119,"Outros cavalos, vivos","Demás caballos, vivos",Other live horses,101,"Cavalos, asininos e muares, vivos","Caballos, asnos, mulos y burdéganos, vivos","Live horses, asses, mules and hinnies",1,Animais vivos,Animales vivos,Live animals,I,Animais vivos e produtos do reino animal,Animales vivos y productos del reino animal,Lives animals; Animal products
3,10120,Asininos e muares vivos,"Asnos, mulos y burdéganos, vivos","Asses, mules and hinnies (live)",101,"Cavalos, asininos e muares, vivos","Caballos, asnos, mulos y burdéganos, vivos","Live horses, asses, mules and hinnies",1,Animais vivos,Animales vivos,Live animals,I,Animais vivos e produtos do reino animal,Animales vivos y productos del reino animal,Lives animals; Animal products
4,10121,Cavalos reprodutores de raça pura,Caballos de raza selecta para reproducción,Pure-bred breeding horses,101,"Cavalos, asininos e muares, vivos","Caballos, asnos, mulos y burdéganos, vivos","Live horses, asses, mules and hinnies",1,Animais vivos,Animales vivos,Live animals,I,Animais vivos e produtos do reino animal,Animales vivos y productos del reino animal,Lives animals; Animal products


In [63]:
def carregar_e_mesclar_arquivos(url_imp_mun_2023: str, url_ncm_sh: str, 
                                chave_principal: str, chave_referencia: str, 
                                sep_principal: str = ',', sep_referencia: str = ';', 
                                encoding: str = 'latin1'):
    
    # Carregar os arquivos dentro da função
    df_imp_mun_2023 = pd.read_csv(url_imp_mun_2023, sep=sep_principal, encoding=encoding)
    df_ncm_sh = pd.read_csv(url_ncm_sh, sep=sep_referencia, encoding=encoding)

    # Garantir que as colunas de chave sejam strings
    df_imp_mun_2023[chave_principal] = df_imp_mun_2023[chave_principal].astype(str)
    df_ncm_sh[chave_referencia] = df_ncm_sh[chave_referencia].astype(str)

    # Realizar o merge
    df_resultado = df_imp_mun_2023.merge(df_ncm_sh, left_on=chave_principal, right_on=chave_referencia, how='left')

    return df_resultado

# Exemplo de uso
df_1 = carregar_e_mesclar_arquivos(
    url_imp_mun_2023=r'\\wsl.localhost\Ubuntu\home\edcarlos\Data-Analytics-Comercio-Exterior\datalake\landing\importacao_mun_2023.csv',
    url_ncm_sh=r'\\wsl.localhost\Ubuntu\home\edcarlos\Data-Analytics-Comercio-Exterior\datalake\raw_data\NCM_SH.csv',
    chave_principal='cd_sh4',
    chave_referencia='CO_SH4'
)

# Exibir as primeiras linhas para verificar
df_1.head()


,cd_sh4,cd_pais,sg_ufmun,cd_mun,kg_liquido,vl_dolarfob,data_importacao_mun,CO_SH6,NO_SH6_POR,NO_SH6_ESP,NO_SH6_ING,CO_SH4,NO_SH4_POR,NO_SH4_ESP,NO_SH4_ING,CO_SH2,NO_SH2_POR,NO_SH2_ESP,NO_SH2_ING,CO_NCM_SECROM,NO_SEC_POR,NO_SEC_ESP,NO_SEC_ING
0,3104,676,go,5305109,35717261,15254781,2023-05-01,310410,"Carnalita e outros sais de potássio, naturais,...","carnalita y demás sales de potasio, naturales...",carnalite and other crude natural potassium salts,3104,"Adubos (fertilizantes) minerais ou químicos, p...",Abonos minerales o químicos potásicos,"Mineral or chemical fertilisers, potassic",31,Adubos (fertilizantes),Abonos,Fertilisers,VI,Produtos das indústrias químicas ou indústrias...,Productos de las industrias químicas o de las ...,Products of the chemical or allied industries
1,3104,676,go,5305109,35717261,15254781,2023-05-01,310420,Cloreto de potássio para uso como fertilizante,"Cloruro de potasio, para su utilización como a...",Potassium chloride for use as fertiliser (excl...,3104,"Adubos (fertilizantes) minerais ou químicos, p...",Abonos minerales o químicos potásicos,"Mineral or chemical fertilisers, potassic",31,Adubos (fertilizantes),Abonos,Fertilisers,VI,Produtos das indústrias químicas ou indústrias...,Productos de las industrias químicas o de las ...,Products of the chemical or allied industries
2,3104,676,go,5305109,35717261,15254781,2023-05-01,310430,Sulfato de potássio,Sulfato de potasio (exc. en tabletas o formas ...,Potassium sulphate (excl. that in pellet or si...,3104,"Adubos (fertilizantes) minerais ou químicos, p...",Abonos minerales o químicos potásicos,"Mineral or chemical fertilisers, potassic",31,Adubos (fertilizantes),Abonos,Fertilisers,VI,Produtos das indústrias químicas ou indústrias...,Productos de las industrias químicas o de las ...,Products of the chemical or allied industries
3,3104,676,go,5305109,35717261,15254781,2023-05-01,310490,Outros adubos ou fertilizantes minerais ou quí...,"Sulfato de magnesio y de potasio, mezclas de c...","Carnallite, sylvite and other crude natural po...",3104,"Adubos (fertilizantes) minerais ou químicos, p...",Abonos minerales o químicos potásicos,"Mineral or chemical fertilisers, potassic",31,Adubos (fertilizantes),Abonos,Fertilisers,VI,Produtos das indústrias químicas ou indústrias...,Productos de las industrias químicas o de las ...,Products of the chemical or allied industries
4,6104,858,sp,3415004,8189,255925,2023-12-01,610411,Tailleurs de malha de lã ou de pêlos finos,trajes sastre de punto de lana o de pelo fino,suits of mesh of wool or fine animal hair,6104,"Fatos de saia-casaco, conjuntos, casacos, vest...","Trajes sastre, conjuntos, chaquetas (sacos), v...","Women's or girls' suits, ensembles, jackets, b...",61,"Vestuário e seus acessórios, de malha","Prendas y complementos (accesorios), de vestir...","Articles of apparel and clothing accessories, ...",XI,Matérias têxteis e suas obras,Materias textiles y sus manufacturas,Textiles and textiles articles


In [65]:
print(df_1.columns)

Index(['cd_sh4', 'cd_pais', 'sg_ufmun', 'cd_mun', 'kg_liquido', 'vl_dolarfob',
       'data_importacao_mun', 'CO_SH6', 'NO_SH6_POR', 'NO_SH6_ESP',
       'NO_SH6_ING', 'CO_SH4', 'NO_SH4_POR', 'NO_SH4_ESP', 'NO_SH4_ING',
       'CO_SH2', 'NO_SH2_POR', 'NO_SH2_ESP', 'NO_SH2_ING', 'CO_NCM_SECROM',
       'NO_SEC_POR', 'NO_SEC_ESP', 'NO_SEC_ING'],
      dtype='object')


In [66]:
df_1 = df_1.drop(columns=['cd_sh4', 'CO_SH6', 'NO_SH6_POR', 'NO_SH6_ESP',
       'NO_SH6_ING', 'CO_SH4', 'NO_SH4_ESP', 'NO_SH4_ING',
       'CO_SH2', 'NO_SH2_POR', 'NO_SH2_ESP', 'NO_SH2_ING', 'CO_NCM_SECROM',
       'NO_SEC_POR', 'NO_SEC_ESP', 'NO_SEC_ING'])
df_1.head()

,cd_pais,sg_ufmun,cd_mun,kg_liquido,vl_dolarfob,data_importacao_mun,NO_SH4_POR
0,676,go,5305109,35717261,15254781,2023-05-01,"Adubos (fertilizantes) minerais ou químicos, p..."
1,676,go,5305109,35717261,15254781,2023-05-01,"Adubos (fertilizantes) minerais ou químicos, p..."
2,676,go,5305109,35717261,15254781,2023-05-01,"Adubos (fertilizantes) minerais ou químicos, p..."
3,676,go,5305109,35717261,15254781,2023-05-01,"Adubos (fertilizantes) minerais ou químicos, p..."
4,858,sp,3415004,8189,255925,2023-12-01,"Fatos de saia-casaco, conjuntos, casacos, vest..."


In [67]:
def carregar_e_mesclar_2(df_1: pd.DataFrame, caminho_referencia: str, 
                       chave_base: str, chave_referencia: str, 
                       sep_referencia: str = ';', encoding: str = 'latin1') -> pd.DataFrame:

    # Carregar o arquivo CSV de referência
    df_referencia = pd.read_csv(caminho_referencia, sep=sep_referencia, encoding=encoding)

    # Garantir que as colunas usadas no merge sejam strings
    df_1[chave_base] = df_1[chave_base].astype(str)
    df_referencia[chave_referencia] = df_referencia[chave_referencia].astype(str)

    # Realizar o merge
    df_resto = df_1.merge(df_referencia, left_on=chave_base, right_on=chave_referencia, how='left')
    
    return df_resto

# Caminho do arquivo de países
arquivo_pais = r'\\wsl.localhost\Ubuntu\home\edcarlos\Data-Analytics-Comercio-Exterior\datalake\raw_data\PAIS.csv'

# Aplicar a função na base principal (df_1 já existe previamente)
df_1 = carregar_e_mesclar_2(df_1, arquivo_pais, chave_base='cd_pais', chave_referencia='CO_PAIS')

# Exibir as primeiras linhas
df_1.head()



,cd_pais,sg_ufmun,cd_mun,kg_liquido,vl_dolarfob,data_importacao_mun,NO_SH4_POR,CO_PAIS,CO_PAIS_ISON3,CO_PAIS_ISOA3,NO_PAIS,NO_PAIS_ING,NO_PAIS_ESP
0,676,go,5305109,35717261,15254781,2023-05-01,"Adubos (fertilizantes) minerais ou químicos, p...",676,643,RUS,Rússia,Russia,Rusia
1,676,go,5305109,35717261,15254781,2023-05-01,"Adubos (fertilizantes) minerais ou químicos, p...",676,643,RUS,Rússia,Russia,Rusia
2,676,go,5305109,35717261,15254781,2023-05-01,"Adubos (fertilizantes) minerais ou químicos, p...",676,643,RUS,Rússia,Russia,Rusia
3,676,go,5305109,35717261,15254781,2023-05-01,"Adubos (fertilizantes) minerais ou químicos, p...",676,643,RUS,Rússia,Russia,Rusia
4,858,sp,3415004,8189,255925,2023-12-01,"Fatos de saia-casaco, conjuntos, casacos, vest...",858,704,VNM,Vietnã,Vietnam,Viet Nam


In [68]:
print(df_1.columns)

Index(['cd_pais', 'sg_ufmun', 'cd_mun', 'kg_liquido', 'vl_dolarfob',
       'data_importacao_mun', 'NO_SH4_POR', 'CO_PAIS', 'CO_PAIS_ISON3',
       'CO_PAIS_ISOA3', 'NO_PAIS', 'NO_PAIS_ING', 'NO_PAIS_ESP'],
      dtype='object')


In [70]:
df_2 = df_1.drop(columns=['cd_pais', 'CO_PAIS', 'CO_PAIS_ISON3',
       'CO_PAIS_ISOA3', 'NO_PAIS_ING', 'NO_PAIS_ESP'])
df_2.head()

,sg_ufmun,cd_mun,kg_liquido,vl_dolarfob,data_importacao_mun,NO_SH4_POR,NO_PAIS
0,go,5305109,35717261,15254781,2023-05-01,"Adubos (fertilizantes) minerais ou químicos, p...",Rússia
1,go,5305109,35717261,15254781,2023-05-01,"Adubos (fertilizantes) minerais ou químicos, p...",Rússia
2,go,5305109,35717261,15254781,2023-05-01,"Adubos (fertilizantes) minerais ou químicos, p...",Rússia
3,go,5305109,35717261,15254781,2023-05-01,"Adubos (fertilizantes) minerais ou químicos, p...",Rússia
4,sp,3415004,8189,255925,2023-12-01,"Fatos de saia-casaco, conjuntos, casacos, vest...",Vietnã


In [71]:
# Caminho do arquivo
caminho_uf_mun = r'\\wsl.localhost\Ubuntu\home\edcarlos\Data-Analytics-Comercio-Exterior\datalake\raw_data\UF_MUN.csv' #foi nomeado dessa forma porque não encontrou o arquivo porque está no wsl2
# Lendo o CSV para um DataFrame
df_uf_mun = pd.read_csv(caminho_uf_mun, sep=';',  encoding='latin1')  # Ou tente 'ISO-8859-1'
# Exibir as primeiras linhas do DataFrame
df_uf_mun.head()

,CO_MUN_GEO,NO_MUN,NO_MUN_MIN,SG_UF
0,2502003,BELEM DO BREJO DO CRUZ,Belém do Brejo do Cruz,PB
1,2502052,BERNARDINO BATISTA,Bernardino Batista,PB
2,2502102,BOA VENTURA,Boa Ventura,PB
3,2502151,BOA VISTA,Boa Vista,PB
4,2502201,BOM JESUS,Bom Jesus,PB


In [72]:
# Função para mesclar os DataFrames já carregados
def mesclar_dataframes(df_base: pd.DataFrame, df_referencia: pd.DataFrame, 
                       chave_base: str, chave_referencia: str) -> pd.DataFrame:
    """
    Realiza a junção (merge) entre a base principal e o DataFrame de referência.
    """
    # Garantir que as colunas usadas no merge sejam strings
    df_base[chave_base] = df_base[chave_base].astype(str)
    df_referencia[chave_referencia] = df_referencia[chave_referencia].astype(str)

    # Realizar o merge
    df_resultante = df_base.merge(df_referencia, left_on=chave_base, right_on=chave_referencia, how='left')
    
    return df_resultante

# Exemplo de aplicação da função

# Já carregados anteriormente
# df2 (base principal) e df_uf_mun (referência) já estão definidos

# Mesclar os DataFrames
df_2 = mesclar_dataframes(df_base=df_2, df_referencia=df_uf_mun, chave_base='cd_mun', chave_referencia='CO_MUN_GEO')

# Exibir as primeiras linhas do DataFrame resultante
df_2.head()



,sg_ufmun,cd_mun,kg_liquido,vl_dolarfob,data_importacao_mun,NO_SH4_POR,NO_PAIS,CO_MUN_GEO,NO_MUN,NO_MUN_MIN,SG_UF
0,go,5305109,35717261,15254781,2023-05-01,"Adubos (fertilizantes) minerais ou químicos, p...",Rússia,5305109,CATALAO,Catalão,GO
1,go,5305109,35717261,15254781,2023-05-01,"Adubos (fertilizantes) minerais ou químicos, p...",Rússia,5305109,CATALAO,Catalão,GO
2,go,5305109,35717261,15254781,2023-05-01,"Adubos (fertilizantes) minerais ou químicos, p...",Rússia,5305109,CATALAO,Catalão,GO
3,go,5305109,35717261,15254781,2023-05-01,"Adubos (fertilizantes) minerais ou químicos, p...",Rússia,5305109,CATALAO,Catalão,GO
4,sp,3415004,8189,255925,2023-12-01,"Fatos de saia-casaco, conjuntos, casacos, vest...",Vietnã,3415004,EMBU,Embu,SP


In [73]:
df_2.columns

Index(['sg_ufmun', 'cd_mun', 'kg_liquido', 'vl_dolarfob',
       'data_importacao_mun', 'NO_SH4_POR', 'NO_PAIS', 'CO_MUN_GEO', 'NO_MUN',
       'NO_MUN_MIN', 'SG_UF'],
      dtype='object')

In [74]:
df_2 = df_2.drop(columns=['cd_mun', 'CO_MUN_GEO', 'NO_MUN', 'SG_UF'])
df_2.head()

,sg_ufmun,kg_liquido,vl_dolarfob,data_importacao_mun,NO_SH4_POR,NO_PAIS,NO_MUN_MIN
0,go,35717261,15254781,2023-05-01,"Adubos (fertilizantes) minerais ou químicos, p...",Rússia,Catalão
1,go,35717261,15254781,2023-05-01,"Adubos (fertilizantes) minerais ou químicos, p...",Rússia,Catalão
2,go,35717261,15254781,2023-05-01,"Adubos (fertilizantes) minerais ou químicos, p...",Rússia,Catalão
3,go,35717261,15254781,2023-05-01,"Adubos (fertilizantes) minerais ou químicos, p...",Rússia,Catalão
4,sp,8189,255925,2023-12-01,"Fatos de saia-casaco, conjuntos, casacos, vest...",Vietnã,Embu


In [75]:
print(df_2.columns)

Index(['sg_ufmun', 'kg_liquido', 'vl_dolarfob', 'data_importacao_mun',
       'NO_SH4_POR', 'NO_PAIS', 'NO_MUN_MIN'],
      dtype='object')


In [ ]:
df_3 = df_2.rename(columns={  
    'sg_ufmun': 'sg_ufmun', 
    'kg_liquido': 'kg_liquido', 
    'vl_dolarfob': 'vl_dolarfob', 
    'data_importacao_mun': 'data_importacao_mun',
    'NO_SH4_POR': 'cd_sh4', 
    'NO_PAIS': 'cd_pais', 
    'NO_MUN_MIN': 'cd_mun'
    
    })
df_3.columns = df_3.columns.str.strip().str.lower().str.replace(' ', '_')
df_3.head()

,sg_ufmun,kg_liquido,vl_dolarfob,data_importacao_mun,cd_sh4,cd_pais,cd_mun
0,go,35717261,15254781,2023-05-01,"Adubos (fertilizantes) minerais ou químicos, p...",Rússia,Catalão
1,go,35717261,15254781,2023-05-01,"Adubos (fertilizantes) minerais ou químicos, p...",Rússia,Catalão
2,go,35717261,15254781,2023-05-01,"Adubos (fertilizantes) minerais ou químicos, p...",Rússia,Catalão
3,go,35717261,15254781,2023-05-01,"Adubos (fertilizantes) minerais ou químicos, p...",Rússia,Catalão
4,sp,8189,255925,2023-12-01,"Fatos de saia-casaco, conjuntos, casacos, vest...",Vietnã,Embu


In [79]:
df_3['cd_pais'] = df_3['cd_pais'].str.lower()
df_3['cd_mun'] = df_3['cd_mun'].str.lower()
df_3.head()

,sg_ufmun,kg_liquido,vl_dolarfob,data_importacao_mun,cd_sh4,cd_pais,cd_mun
0,go,35717261,15254781,2023-05-01,"Adubos (fertilizantes) minerais ou químicos, p...",rússia,catalão
1,go,35717261,15254781,2023-05-01,"Adubos (fertilizantes) minerais ou químicos, p...",rússia,catalão
2,go,35717261,15254781,2023-05-01,"Adubos (fertilizantes) minerais ou químicos, p...",rússia,catalão
3,go,35717261,15254781,2023-05-01,"Adubos (fertilizantes) minerais ou químicos, p...",rússia,catalão
4,sp,8189,255925,2023-12-01,"Fatos de saia-casaco, conjuntos, casacos, vest...",vietnã,embu


In [ ]:
caminho = "/home/edcarlos/Data-Analytics-Comercio-Exterior/datalake/silver/exportacao_mun_2023_m.csv"
df_3.to_csv(caminho, index=False)
print("Arquivo salvo com sucesso!")

Arquivo salvo com sucesso!
